In [203]:
import pickle
import pandas as pd
import numpy as np

In [204]:
DATASET_PATH = './datasets/preprocessed.pkl'
dataset = pd.read_pickle(DATASET_PATH)

In [205]:
dataset = dataset.sample(frac=1, random_state=4)
train_data = dataset[:int(len(dataset)*0.9)]
test_data = dataset[int(len(dataset)*0.9):]
print("Dataset size: ", len(dataset))
print("Test Data size: ", len(test_data))
print("Train Data size: ", len(train_data))

Dataset size:  14000
Test Data size:  1400
Train Data size:  12600


In [206]:

vocabulary = set()
tf = dict()


def make_vocab(sentence):

    for word in sentence:
         vocabulary.add(word)


train_data["comment"].apply(make_vocab)


8756     None
5474     None
11242    None
7820     None
7909     None
         ... 
10142    None
8828     None
11554    None
11609    None
904      None
Name: comment, Length: 12600, dtype: object

In [207]:
# import TfidfVectorizer from scikit-learn
# from sklearn.feature_extraction.text import TfidfVectorizer
# train_data["comment"] = train_data["comment"].apply(lambda x: " ".join(x))
# vectorizer = TfidfVectorizer()
# vec = vectorizer.fit_transform(train_data["comment"])
train_data["comment"].apply(make_vocab)



8756     None
5474     None
11242    None
7820     None
7909     None
         ... 
10142    None
8828     None
11554    None
11609    None
904      None
Name: comment, Length: 12600, dtype: object

Check the difference between the set and dict below

In [208]:
type(vocabulary)

set

In [209]:
# type(vectorizer.vocabulary_)

In [210]:
# def compute_tf(term,document):
#     return document.count(term)/len(document)

# train_data = train_data.sort_index()

# for index,row in train_data.iterrows() :
#     print(index)
#     for word in vocabulary: 
#         for word in row["comment"]:
#             train_data["tf"] = compute_tf(word,row["comment"])
#     percentage = 100*(index+1)/len(train_data)
#     if percentage % 1 == 0:
#         print(f"Processing iteration {index+1}/{len(train_data)} ({percentage:.0f}%)")


In [211]:
# import numpy as np 

# def compute_tf(term, document):
#     return document.count(term) / len(document)

# # Define a function to calculate inverse document frequency (IDF) of a term in a corpus
# def compute_idf(term, corpus):
#     N = len(corpus)
#     df = sum(1 for document in corpus if term in document)
#     return np.log(N / df)

# # Load the dataset

# # Create a list of all unique words in the dataset
# unique_words = list(set(word for document in train_data['comment'] for word in document))

# # Calculate the TF-IDF matrix
# tfidf_matrix = []

# for i,document in enumerate(train_data['comment']):
#     tfidf_vector = []
#     print(i)
#     for term in unique_words:
#         tf = compute_tf(term, document)
#         idf = compute_idf(term, train_data['comment'])
#         tfidf = tf * idf
#         tfidf_vector.append(tfidf)
#     tfidf_matrix.append(tfidf_vector)

# # Convert the TF-IDF matrix to a NumPy array
# tfidf_array = np.array(tfidf_matrix)

In [212]:
# sk_vocab = set(vectorizer.vocabulary_)
# len(vocabulary - sk_vocab)

In [213]:
word_index = dict()
word_idf = dict()
def map_word_index(vocabulary):
    for index,word in enumerate(vocabulary):
        word_index[word] = index
        

In [215]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 69
model name	: Intel(R) Core(TM) i7-4510U CPU @ 2.00GHz
stepping	: 1
microcode	: 0xffffffff
cpu MHz		: 2593.993
cache size	: 4096 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology cpuid pni pclmulqdq vmx ssse3 fma cx16 pcid sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp tpr_shadow vnmi ept vpid ept_ad fsgsbase bmi1 avx2 smep bmi2 erms invpcid xsaveopt flush_l1d arch_capabilities
vmx flags	: vnmi invvpid ept_x_only ept_ad ept_1gb tsc_offset vtpr ept vpid unrestricted_guest
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit srbds
bogomips	: 5187.

In [217]:
from tqdm import tqdm
def compute_idf(word,dataset):
    count = 0
    for comment in dataset["comment"]:
        if word in comment:
            count += 1
    return np.log(len(dataset["comment"])/count)

for index, word in tqdm(enumerate(vocabulary)):
    # print(index)
    word_idf[word] = compute_idf(word,train_data)
    percentage = 100*(index+1)/len(vocabulary)
    if percentage % 1 == 0:
        print(f"Processing iteration {index+1}/{len(vocabulary)} ({percentage:.0f}%)")




13783it [04:00, 57.42it/s]

Processing iteration 13783/13783 (100%)


In [ ]:

def compute_tf(word,comment):
    freq = 0 
    for part in comment:        
        if word == part:
            freq += 1
    return freq/len(comment)
    

def fit_tfidf(comment,index,dataset_len):
    vec = np.zeros(len(vocabulary))
    for word in comment:
        # if word in vocabulary:
            tf = compute_tf(word,comment)
            vec[word_index[word]] = tf*word_idf[word]
    print(index)
    
    percentage = 100*(index+1)/dataset_len
    if percentage % 2 == 0:
        print(f"Processing iteration {index+1}/{dataset_len} ({percentage:.0f}%)")

    return vec

In [ ]:
map_word_index(vocabulary)
train_data = train_data.sort_index()
train_data["vectorized"] = train_data.apply(lambda row: fit_tfidf(row["comment"],row.name,len(train_data)),axis=1)

0
1
2
3
4
5
7
8
9
10
11
13
14
15
16
17
18
19
20
21
22
24
25
26
27
28
29
30
31
33
34
35
36
37
38
39
40
41
42
43
44
45
47
48
49
50
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
85
87
88
89
90
91
92
93
94
95
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
113
114
115
116
117
118
119
120
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172


KeyboardInterrupt: 